#  Overview

This notebook provides step-by-step details on the calculations for Criterion B.

# Setup

Import Python modules

In [ ]:
import math
import os

import ee
from gee_redlist.ee_rle import make_eoo, area_km2, get_aoo_grid_projection
import geopandas as gpd
import ipywidgets as widgets
from lonboard import Map, PolygonLayer, BitmapTileLayer

from gee_redlist.ee_auth import initialize_ee
from gee_redlist.ee_rle import load_yaml

Initialize Earth Engine


In [ ]:
initialize_ee(project=os.environ['GOOGLE_CLOUD_PROJECT'])

# Analysis

In [ ]:
# Define the ecosystem that this notebook is analyzing
ecosystem_code = 'MMR-T1.1.1'

In [ ]:
# Load the country config
country_config_path = os.environ['VSCODE_CWD'] + '/config/country_config.yaml'
country_config = load_yaml(country_config_path)
print(f'{country_config = }')

gee_project_path = country_config['gee_project_path']

In [ ]:
# Extract the class info for the ecosystem
class_info = [x for x in country_config['classified_image']['classes'] if x['code'] == ecosystem_code][0]
print(f'{class_info = }')

ecosystem_image = {
    'asset_id': f"{gee_project_path}/{ecosystem_code}/{class_info['id']}",
    'pixel_value': class_info['id']
}
print(f'{ecosystem_image = }')

In [ ]:
classified_image_asset_id = f"{country_config['gee_project_path']}/{country_config['classified_image']['asset_id']}"
print(f'{classified_image_asset_id = }')

class_img = (
    ee.Image(classified_image_asset_id)
      .eq(ecosystem_image['pixel_value'])
      .selfMask()
)
print(f'class_img: {class_img.getInfo()}')

In [ ]:
# Commented out until bug is fixed: https://github.com/developmentseed/lonboard/issues/1064
# # Determine the coordinates for viewing the image
# longitude, latitude = class_img.geometry().centroid().getInfo()['coordinates']
# longitude, latitude

# Extent of occurrence (EOO) (subcriterion B1)

Set the scale (in meters) for reducing the image pixels to polygons. Use the image's nominal scale unless is is less than 50 meters per pixel.

In [ ]:
reduction_scale = max(class_img.projection().nominalScale().getInfo(), 50)
reduction_scale

Covert the classified image to vectors.

In [ ]:
ecosystem_polygons = class_img.updateMask(1).reduceToVectors(
    scale=reduction_scale,
    geometry=class_img.geometry(),
    geometryType='polygon',
    maxPixels=1e12,
    bestEffort=False
)

Create a convex hull that encompasses the ecosystem polygons.

In [ ]:
# convexHull() is called twice as a workaround for a bug
# (https://issuetracker.google.com/issues/465490917)
hull = ecosystem_polygons.geometry().convexHull(maxError=1).convexHull(maxError=1)

In [ ]:
aoo_area_km2 = hull.area().multiply(1e-6).getInfo()
print(f'The area of the hull is {aoo_area_km2:.0f} km²')

## Display EOO Layers

## Ecosystem Tiles

In [ ]:
tile_url = class_img.getMapId(
    vis_params={
        'palette': ['blue'],
        'opacity': 0.5
    }
)['tile_fetcher'].url_format

tile_layer = BitmapTileLayer(
    data=tile_url,
    tile_size=256,
    max_requests=-1,
    min_zoom=0,
    max_zoom=19,
)

## Ecosystem Polygons Layer

In [ ]:
ecosystem_polygons_gdf = gpd.GeoDataFrame.from_features(
    ecosystem_polygons.getInfo()['features'],
    crs='EPSG:4326'
)

ecosystem_polygons_layer = PolygonLayer.from_geopandas(
    ecosystem_polygons_gdf,
    get_fill_color=[255, 0, 0, 127],
    stroked=True,
    get_line_width=2,
    get_line_color=[0, 0, 0, 150],
)

## Ecosystem Hull Layer

In [ ]:
hull_gdf = gpd.GeoDataFrame.from_features(
    ee.FeatureCollection(hull).getInfo(),
    crs='EPSG:4326'
)

hull_layer = PolygonLayer.from_geopandas(
    hull_gdf,
    get_fill_color=[0, 0, 255, 63],
    stroked=True,
    get_line_width=200,
    get_line_color=[0, 0, 0, 150],
)

Display the map.

In [ ]:
m = Map(
    layers=[
        ecosystem_polygons_layer,
        tile_layer,
        hull_layer
    ],
    controls=[]
)

checkbox_hull = widgets.Checkbox(value=True, description='Hull')
checkbox_tiles = widgets.Checkbox(value=False, description='Tiles')
checkbox_ecosystem_polygons = widgets.Checkbox(value=True, description='Gridcells')

# Link checkboxes to layer visibility
widgets.link((checkbox_hull, 'value'), (hull_layer, 'visible'))
widgets.link((checkbox_tiles, 'value'), (tile_layer, 'visible'))
widgets.link((checkbox_ecosystem_polygons, 'value'), (ecosystem_polygons_layer, 'visible'))

controls = widgets.VBox([checkbox_tiles, checkbox_ecosystem_polygons, checkbox_hull])
display(widgets.HBox([controls, m]))

## Verify that the step-by-step results are consistent

In [ ]:
# Direct call to `make_eoo()`
aoo_area_km2_direct_call = area_km2(make_eoo(class_img)).getInfo()
print(f'EOO area: {aoo_area_km2_direct_call:.0f} km²')

In [ ]:
# Assert the two values are close to each other.
assert math.isclose(aoo_area_km2, aoo_area_km2_direct_call, abs_tol=1e-4)

# Area of Occupancy (AOO) (subcriterion B2)

The protocol for this adjustment includes the following steps:

> - Intersect AOO grid with the ecosystem’s distribution map.
> - Calculate extent of the ecosystem type in each grid cell (‘area’) and sum these areas to obtain the total ecosystem area (‘total area’).
> - Arrange grid cells in ascending order based on their area (smaller first).
> - Calculate accumulated sum of area per cell (‘cumulative area’).
> - Calculate ‘cumulative proportion’ by dividing ‘cumulative area’ by ‘total area’ (cumulative proportion takes values between 0 and 1).
> - Calculate AOO by counting the number of cells with a ‘cumulative proportion’ greater than 0.01 (i.e. exclude cells that in combination account for up to 1% of the total mapped extent of the ecosystem type).

## Intersect AOO grid with the ecosystem’s distribution map

Load the AOO grid projection

In [ ]:
aoo_grid_proj = get_aoo_grid_projection()

aoo_grid_proj.getInfo()

Extract the grid scale parameters

In [ ]:
aoo_x_scale, _, _, _, aoo_y_scale, _ = aoo_grid_proj.getInfo()['transform']
print(f'{aoo_x_scale = } meters')
print(f'{aoo_y_scale = } meters')

> - Create an Earth Engine feature collection of AOO grid cells that intersect with the ecosystem, and calculate the fractional coverage of the ecosystem within the grid cell.

In [ ]:
fractional_coverage_fc = class_img.unmask().reduceRegions(
  collection=class_img.geometry().coveringGrid(aoo_grid_proj),
  reducer=ee.Reducer.mean(),
).filter(ee.Filter.gt('mean', 0))

# Convert the Earth Engine feature collection to a GeoPandas GeoDataFrame.
fractional_coverage_gdf = ee.data.computeFeatures({
    "expression": fractional_coverage_fc,
    "fileFormat": "GEOPANDAS_GEODATAFRAME",
})
fractional_coverage_gdf.rename(columns={"mean": "coverage"}, inplace=True)

In [ ]:
len(fractional_coverage_gdf)

## Calculate grid cell area and the total ecosystem area

> - Calculate extent of the ecosystem type in each grid cell (‘area’) and sum these areas to obtain the total ecosystem area (‘total area’).

In [ ]:
fractional_coverage_gdf['area'] = fractional_coverage_gdf['coverage'] * aoo_x_scale * aoo_y_scale

fractional_coverage_gdf.sort_values(by="area")[0:4]

In [ ]:
total_area_km2 = fractional_coverage_gdf['area'].sum() / 1e6
print(f'Total ecosystem area: {total_area_km2:.0f} km²')

## Calculate cumulative area in ordered cells

> - Arrange grid cells in ascending order based on their area (smaller first).
> - Calculate accumulated sum of area per cell (‘cumulative area’).

In [ ]:
fractional_coverage_gdf = fractional_coverage_gdf.sort_values(by="area")
fractional_coverage_gdf["cumulative_area"] = fractional_coverage_gdf["area"].cumsum()

fractional_coverage_gdf.sort_values(by="area").head()

## Calculate the cumulative proportion

> - Calculate ‘cumulative proportion’ by dividing ‘cumulative area’ by ‘total area’ (cumulative proportion takes values between 0 and 1).

In [ ]:
fractional_coverage_gdf["cumulative_proportion"] = fractional_coverage_gdf["cumulative_area"] / 1e6 / total_area_km2

fractional_coverage_gdf

## Calculate AOO

> - Calculate AOO by counting the number of cells with a ‘cumulative proportion’ greater than 0.01 (i.e. exclude cells that in combination account for up to 1% of the total mapped extent of the ecosystem type).

In [ ]:
aoo_grid_cells = fractional_coverage_gdf[fractional_coverage_gdf["cumulative_proportion"] > 0.01]
print(f"AOO (number of cells with cumulative proportion > 0.01): {len(aoo_grid_cells)}")

In [ ]:
aoo_grid_cells_dropped = fractional_coverage_gdf[fractional_coverage_gdf["cumulative_proportion"] <= 0.01]

## Display the layers

In [ ]:
aoo_grid_cells_layer = PolygonLayer.from_geopandas(
    aoo_grid_cells,
    get_fill_color=[0, 255, 0, 63],
)

aoo_grid_cells_layer_dropped = PolygonLayer.from_geopandas(
    aoo_grid_cells_dropped,
    get_fill_color=[255, 0, 0, 63],
)

m = Map(
    layers=[
        tile_layer,
        aoo_grid_cells_layer,
        aoo_grid_cells_layer_dropped
    ],
    controls=[]
)

controls = widgets.HBox([])
display(controls, m)

## Verify that the step-by-step results are consistent

In [ ]:
# Direct call to `make_aoo()`
aoo_cells_direct_call = make_aoo(class_img)

print(f'AOO: {aoo_grid_cell_count_direct_call} grid cells')

In [ ]:
# assert aoo_grid_cell_count == aoo_grid_cell_count_direct_call

# Criterion B Summary

In [ ]:
print(f'As shown in the preceding sections, '
      f'AOO and EOO were measured as '
      f'{aoo_grid_cell_count} 10 x 10 km grid cells '
      f'and {aoo_area_km2:.0f} km², respectively.')